# Emotion Recognition Training with FER-2013 Dataset
This notebook trains a CNN model for facial emotion recognition using the FER-2013 dataset from Kaggle.

**Dataset Source**: https://www.kaggle.com/datasets/msambare/fer2013

In [ ]:
# Import required libraries
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
# Explore the dataset structure
data_dir = './dataset/emotion'
print("Dataset structure:")
for root, dirs, files in os.walk(data_dir):
    level = root.replace(data_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files[:5]:  # Show only first 5 files
        print(f"{subindent}{file}")
    if len(files) > 5:
        print(f"{subindent}... and {len(files) - 5} more files")

In [ ]:
# Define emotion classes (using all 7 emotions from FER-2013 dataset)
emotion_classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
num_classes = len(emotion_classes)

print(f"Training with {num_classes} emotion classes: {emotion_labels}")
print("Using full FER-2013 dataset with all 7 emotion categories")

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data generator (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# Load data from directories (auto-detect all emotion classes)
train_generator = train_datagen.flow_from_directory(
    directory="./dataset/emotion/train",
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    directory="./dataset/emotion/test",
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Print detected classes and update our labels accordingly
detected_classes = list(train_generator.class_indices.keys())
print(f"Detected emotion classes: {detected_classes}")
print(f"Number of training samples: {train_generator.n}")
print(f"Number of validation samples: {validation_generator.n}")

# Update num_classes based on actual detected classes
num_classes = len(detected_classes)
print(f"Updated number of classes: {num_classes}")

In [ ]:
# Build CNN model for emotion recognition
model = Sequential()

# Convolutional layers
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))

# Flatten
model.add(Flatten())

# Fully-connected layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Display model architecture
model.summary()

In [ ]:
# Compile the model
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping callback
early_stopping = EarlyStopping(
    patience=3, 
    monitor='val_loss', 
    restore_best_weights=True
)

In [ ]:
# Train the model
print("Starting model training...")
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // validation_generator.batch_size,
    callbacks=[early_stopping],
    verbose=1
)
print("Training completed!")

In [ ]:
# Evaluate the model
score = model.evaluate(validation_generator, verbose=0)
print(f'Test Loss: {score[0]:.4f}')
print(f'Test Accuracy: {score[1]:.4f}')

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], 'r', label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Save the trained model
model.save('Emotion1.h5')
print("Model saved as 'Emotion1.h5'")

In [ ]:
# Test the model with a sample image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load the saved model
model_CNN = load_model('Emotion1.h5')

# Define prediction labels (auto-detect from dataset)
# Note: Order should match the training class indices
try:
    # Try to load class indices from a saved file or use default
    predict_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    print(f"Using prediction labels: {predict_labels}")
except:
    predict_labels = ['Emotion_0', 'Emotion_1', 'Emotion_2', 'Emotion_3', 'Emotion_4', 'Emotion_5', 'Emotion_6']
    print(f"Using default labels: {predict_labels}")

# Function to predict emotion from image path
def predict_emotion(image_path):
    try:
        # Load and preprocess image
        img = load_img(image_path, target_size=(150, 150))
        plt.figure(figsize=(6, 6))
        plt.imshow(img)
        plt.title('Input Image')
        plt.axis('off')
        plt.show()
        
        # Convert to array and normalize
        img_array = img_to_array(img)
        img_array = img_array.reshape(1, 150, 150, 3)
        img_array = img_array.astype('float32') / 255.0
        
        # Make prediction
        prediction = model_CNN.predict(img_array)
        predicted_class = np.argmax(prediction, axis=-1)[0]
        confidence = np.max(prediction)
        
        print(f"Predicted Emotion: {predict_labels[predicted_class]}")
        print(f"Confidence: {confidence:.2%}")
        
        return predict_labels[predicted_class], confidence
    except Exception as e:
        print(f"Error processing image: {e}")
        return None, None

# Example usage (uncomment and provide a valid image path)
# predict_emotion('./dataset/emotion/test/happy/sample_image.jpg')